# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [3]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data()
print(x.shape, y.shape)

(1000, 3) (1000,)


In [4]:
def split_data(X,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    
  np.random.seed(0) # To demonstrate that if we use the same seed value twice, we will get the same random number twice
  n = int(len(X)*train_size)
  indices = np.arange(len(X))
  np.random.shuffle(indices)
  train_idx = indices[: n]
  test_idx = indices[n:]
  X_train, y_train = X[train_idx], y[train_idx]
  X_test, y_test = X[test_idx], y[test_idx]
  
  return X_train, X_test, y_train, y_test


In [5]:
X_train, X_test, y_train, y_test= split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [6]:
def covariance(x, mu):
  
  
  cov = ((x-mu).T@(x-mu))/(x.shape[0]-1)
  

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  return cov

In [7]:
covariance(X_train,np.mean(X_train,axis = 0))

array([[1.81780125, 0.00278495, 1.00021288],
       [0.00278495, 0.98802231, 0.04507526],
       [1.00021288, 0.04507526, 1.73006042]])

In [8]:
cov = np.zeros((X_train.shape[1],X_train.shape[1]))
for i in range(X_train.shape[1]):
  mu_i = np.mean(X_train.T[i])
  for j in range(X_train.shape[1]):
    mu_j = np.mean(X_train.T[j])
    cov[i,j] = (np.sum(i-mu_i*j-mu_j))/(X_train.shape[0]-1)

print(cov)





[[-4.94098787e-05 -1.31261365e-03 -1.73998842e-04]
 [ 1.20215458e-03 -1.27484309e-03 -1.35002218e-03]
 [ 2.45371903e-03  1.16474605e-03  2.27759166e-03]]


In [9]:
np.mean(X_train,axis = 0)

array([0.03947849, 1.00929982, 0.06006809])

In [10]:
cov= np.cov(X_train,rowvar=0)
cov

array([[1.81780125, 0.00278495, 1.00021288],
       [0.00278495, 0.98802231, 0.04507526],
       [1.00021288, 0.04507526, 1.73006042]])

In [19]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu = None
    self.phi = None
    self.sigma =None
    
  def fit(self,x,y):
    k= len(np.unique(y))# Number of class.
    d = x.shape[1]  # input dim
    m=  x.shape[0]# Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(k)# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for i in range(k):
      self.phi[i] = len(y[y==i])/x.shape[0]
      self.mu[i] = np.mean(x[y==i],axis = 0)
      self.sigma[i] = covariance(x[y==i],self.mu[i])

    


  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x
    d= x.shape[1]
    k= self.mu.shape[0]
    k_class = np.zeros((x.shape[0],k))
    constant = (2*np.pi)**(d/2)
    for i in range(k):
      for j in range(x.shape[0]):
        k_class[j,i]= np.exp(-0.5*((x[j]-self.mu[i]).T@np.linalg.inv(self.sigma[i])@(x[j]-self.mu[i]).T))/(constant*(np.linalg.det(self.sigma[i])**0.5))
        #np.exp(-0.5*((x[j]-self.mu[i]).T@np.linalg.inv(self.sigma)@(x[j]-self.mu[i]).T))/(constant*(np.linalg.det(self.sigma[i])**0.5)) # Number of classes we have in our case it's k = 2
    return k_class
    ## START THE LEARNING: estimate mu, phi and sigma.

  def predict(self,x):
    predict = self.predict_proba(x) * self.phi
    y = np.argmax(predict, axis =1)

    #y = np.where(predict[:,0]>predict[:,1],0,1)
    return y

    
  
  def accuracy(self, y, ypreds):
    acc = np.mean(y == ypreds)

    return acc

In [20]:
model= GDA()
model.fit(X_train,y_train)

In [21]:
yproba= model.predict_proba(X_test)
yproba

array([[1.15206101e-070, 5.76116478e-002],
       [4.65661320e-021, 1.96407370e-002],
       [1.26397479e-001, 2.50138981e-004],
       [1.50802026e-092, 4.55699996e-002],
       [1.94935492e-105, 3.28718488e-002],
       [2.16014250e-053, 1.14204868e-001],
       [1.75720703e-001, 1.51958419e-002],
       [8.63916128e-013, 9.95302009e-002],
       [2.62908089e-001, 1.04141218e-005],
       [1.38640584e-004, 3.13742261e-002],
       [1.35545567e-064, 1.21383501e-002],
       [5.27411638e-016, 5.24941943e-002],
       [1.88325934e-036, 1.14483512e-002],
       [1.06667981e-016, 2.09085891e-002],
       [2.51835475e-002, 1.91393407e-008],
       [2.40580939e-070, 3.37322856e-002],
       [4.02488598e-051, 5.85473852e-004],
       [4.27261386e-057, 1.10655762e-001],
       [2.89469546e-008, 2.77792162e-002],
       [3.58551047e-061, 9.74708617e-002],
       [1.52075882e-088, 1.73417555e-002],
       [2.06884411e-001, 2.66507914e-003],
       [1.09924114e-059, 7.40307971e-002],
       [1.1

In [22]:
ypreds= model.predict(X_test)
ypreds


array([1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0])

In [23]:
model.accuracy(y_test, ypreds)

0.98

In [16]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((len(x),1))
    x = np.hstack((one,x))
    return x
    #### END CODE ####

  def sigmoid(self, x):
    ##### WRITE YOUR CODE HERE ####
    sig = 1/(1+np.exp(-x@self.w))
    return sig
    #### END CODE ####

  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -(np.mean((y_true@np.log(y_pred))+(1-y_true)@np.log(1-y_pred)))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = np.where(probas>=0.5,1,0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x = self.add_ones(x)
    # reshape y if needed
    #y = y_train.reshape(len(y_train),-1)
    # Initialize w to zeros vector >>> (x.shape[1])
    self.w = np.zeros((x.shape[1]))
    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-x.T@(y-ypred))

      #update rule
      self.w =self.w - self.lr*dl
      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)

      self.weight.append(self.w)
      self.train_losses.append(loss)

      if epoch%100 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true == y_pred)
    return acc
    #### END CODE ####

In [17]:
model = LogisticRegression(0.01,n_epochs=10000)
model.fit(X_train,y_train)

loss for epoch 0  : 179.05196257302123
loss for epoch 100  : 109.17589031411237
loss for epoch 200  : 109.17409135711719
loss for epoch 300  : 109.17408864556477
loss for epoch 400  : 109.17408864138599
loss for epoch 500  : 109.17408864137954
loss for epoch 600  : 109.17408864137954
loss for epoch 700  : 109.17408864137954
loss for epoch 800  : 109.17408864137953
loss for epoch 900  : 109.17408864137954
loss for epoch 1000  : 109.17408864137954
loss for epoch 1100  : 109.17408864137954
loss for epoch 1200  : 109.17408864137954
loss for epoch 1300  : 109.17408864137954
loss for epoch 1400  : 109.17408864137954
loss for epoch 1500  : 109.17408864137954
loss for epoch 1600  : 109.17408864137954
loss for epoch 1700  : 109.17408864137954
loss for epoch 1800  : 109.17408864137954
loss for epoch 1900  : 109.17408864137954
loss for epoch 2000  : 109.17408864137954
loss for epoch 2100  : 109.17408864137954
loss for epoch 2200  : 109.17408864137954
loss for epoch 2300  : 109.17408864137954
loss

In [18]:
ypred_train = model.predict(X_train)
acc = model.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model.predict(X_test)
acc = model.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 0.96
 
The test accuracy is: 0.95
